In [3]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:///song_list_v2.db', echo=False)

In [5]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [8]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']
    


get_artist_id('54flyrjcdnQdco7300avMJ')

'1dfeR4HaWDbWqFHLkxsg1d'

In [9]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']


get_genres('1dfeR4HaWDbWqFHLkxsg1d')

['glam rock', 'rock']

# Inserts Genre Value into Genres Column

In [ ]:
genres_list = []
for item in engine.execute("SELECT songid FROM features").fetchall()[515122:]:    
    songid = item.values()[0]
    artistid = get_artist_id(songid)
    genres = get_genres(artistid)
    try:
        engine.execute(f'UPDATE features SET genres = \"{genres}\" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print("couldn't insert")
        continue

In [13]:
# null = []
# for genre in genres_list:
#     if genre == []:
#         null.append(genre)

# Convert to PD DF

In [14]:
con = sqlite3.connect("./song_list_v2.db")
df = pd.read_sql_query("SELECT * from features", con)

In [28]:
df.isnull().sum()

index                  0
acousticness           0
artist                 0
danceability           0
duration_ms            0
energy                 0
instrumentalness       0
key                    0
liveness               0
loudness               0
mode                   0
songid                 0
speechiness            0
tempo                  0
time_signature         0
track                  1
valence                0
genres              1468
dtype: int64

# Create List of all Null Values for Investigation

In [31]:
nan_rows = df[df['genres'].isnull()].index.tolist()

In [33]:
len(nan_rows)

1468

In [37]:
df.loc[nan_rows[0]]

index                                  125
acousticness                         0.785
artist                   Dora The Explorer
danceability                         0.766
duration_ms                          48787
energy                               0.226
instrumentalness                         0
key                                      9
liveness                             0.165
loudness                            -5.875
mode                                     1
songid              4LisLrspORq8IagdLHWkf7
speechiness                          0.055
tempo                                93.05
time_signature                           4
track                The Itsy Bitsy Spider
valence                              0.877
genres                                None
Name: 125, dtype: object

In [ ]:
genres_list = []
for item in engine.execute("SELECT songid FROM features").fetchall()[515122:]:    
    songid = item.values()[0]
    artistid = get_artist_id(songid)
    genres = get_genres(artistid)
    try:
        engine.execute(f'UPDATE features SET genres = \"{genres}\" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print("couldn't insert")
        continue